In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qreservoirpy import QReservoir, Layers, utilities

## Initialization

In [ ]:
n_qubits = 4
SHOTS = 10000

timeseries = []
encoder = {} 

In [ ]:
def analyze_func(res):
    pass

def build_method(circuit, timestep, encoding):
    pass

qres = QReservoir(qubits=n_qubits, layers=[
    Layers.Reset(),
    Layers.Timeseries(build_method=build_method, encoding = encoder)
], analyze_function=analyze_func)

## Running

In [ ]:
states = qres.run(
    timeseries=timeseries, 
    shots=SHOTS
)

In [ ]:
WARMUP = 0.1 # 10%
warmup_idx = int(len(states) * WARMUP)

xstates = states[:-1][warmup_idx:]
target = timeseries[1:][warmup_idx:]
utilities.result_plotter(xstates, target)

## Training

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler as Preprocessor

from sklearn.linear_model import RidgeClassifier as Classifier

X_train, X_test, y_train, y_test = train_test_split(xstates, target, test_size=0.33)

In [ ]:
model = make_pipeline(Preprocessor(), Classifier())
model.fit(X_train, y_train)
score = model.score(X_test, y_test)
print(f'Score of model: {score}')

## Predicting

In [ ]:
predict_from = timeseries[-10:]
num_preds = 20


pred_states, preds = qres.predict(
    num_pred=num_preds,
    model=model,
    shots=SHOTS,
    from_series=predict_from
)


In [ ]:
true_values = []

pred_plot = np.append(predict_from, preds)
true_plot = np.append(predict_from, true_values)

plt.plot(pred_plot, label="Predictions")
plt.plot(true_plot, label="True timeseries")
plt.legend()